In [138]:
# common imports
import numpy as np
import pandas as pd
import joblib, glob
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning imports


# display setup
pd.set_option("display.max_columns", None)  # the None parameter displays unlimited columns
plt.style.use('seaborn')  # for plots

# 1. Getting the Data

> Each participant has 3 csv files, one for each alignment state.
>
> Taking a look at one file for each state should help gain insight to which dataframe manipulations are necessary
> before inserting it into a machine learning algorithm.

In [304]:
# read right hand file
handright = pd.read_csv(r"extraFiles/HandRight.csv")

In [305]:
handright.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,Roll,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,128.2883,15478,right,1,15.67167,226.2064,20.26470,34.26665,-195.214700,-35.043020,0.270987,-0.017635,-0.277800,46.88831,207.3548,86.23589,203.8337,49.33507,237.4553,0.0,0.206389,0.0
1,128.3048,15480,right,1,16.24119,222.9389,20.11781,36.16443,-180.093900,-4.052799,0.245085,-0.014770,-0.277763,47.54861,205.6568,86.42250,203.9553,47.88272,238.4482,0.0,0.243050,0.0
2,128.3218,15482,right,1,16.72461,220.9709,19.94972,26.35298,-95.088190,-11.157620,0.230796,-0.008779,-0.278792,48.15030,204.5182,86.35149,204.6231,45.35921,236.8544,0.0,0.281134,0.0
3,128.3384,15484,right,1,17.09016,219.7124,19.67679,19.46804,-61.755180,-21.303780,0.217681,-0.010759,-0.279768,48.64175,203.9989,86.16056,204.9939,44.18692,236.0927,0.0,0.277691,0.0
4,128.3551,15486,right,1,17.46017,219.3652,19.13128,21.94964,-8.355943,-35.081790,0.220696,-0.017380,-0.278629,48.93135,203.3777,85.59017,204.5952,42.68003,235.2896,0.0,0.283008,0.0


In [306]:
handright.shape

(4812, 22)

In [307]:
handright.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4812 entries, 0 to 4811
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Time             4812 non-null   float64
 1    Frame ID        4812 non-null   int64  
 2    Hand Type       4812 non-null   object 
 3    # hands         4812 non-null   int64  
 4    Position X      4812 non-null   float64
 5    Position Y      4812 non-null   float64
 6    Position Z      4812 non-null   float64
 7    Velocity X      4812 non-null   float64
 8    Velocity Y      4812 non-null   float64
 9    Velocity Z      4812 non-null   float64
 10   Pitch           4812 non-null   float64
 11   Roll            4812 non-null   float64
 12   Yaw             4812 non-null   float64
 13   Wrist Pos X     4428 non-null   float64
 14   Wrist Pos Y     4428 non-null   float64
 15   Wrist Pos Z     4428 non-null   float64
 16   Elbow pos X     4428 non-null   float64
 17   Elbow Pos Y  

In [308]:
handright.isna().sum()

Time                 0
 Frame ID            0
 Hand Type           0
 # hands             0
 Position X          0
 Position Y          0
 Position Z          0
 Velocity X          0
 Velocity Y          0
 Velocity Z          0
 Pitch               0
 Roll                0
 Yaw                 0
 Wrist Pos X       384
 Wrist Pos Y       384
 Wrist Pos Z       384
 Elbow pos X       384
 Elbow Pos Y       384
 Elbow Pos Z       384
 Grab Strenth        0
 Grab Angle          0
 Pinch Strength      0
dtype: int64

In [309]:
handright[" # hands"].value_counts()

1    4732
2      80
Name:  # hands, dtype: int64

In [310]:
handright[" Hand Type"].value_counts()

right    4738
left       74
Name:  Hand Type, dtype: int64

> Features in the DataFrame:
1. Time: Second and milliseconds of recorded movement
2. Frame ID: Identification number of detected movement
3. Hand Type: Right or left hand
4. Number hands: Hands detected in the frame/movement
5. Position X: Center position of the palm from the leap origin, millimeters
6. Position Y:
7. Position Z:
8. Velocity X: Rate of change
9. Velocity Y:
10. Velocity Z:
11. Pitch:
12. Roll:
13. Yaw:
14. Wrist Pos X:
15. Wrist Pos Y:
16. Wrist Pos Z:
17. Elbow pos X:
18. Elbow pos Y:
19. Elbow pos Z:
20. Grab Strength:
21. Grab Angle:
22. Pinch Strength:

Clearly just by looking at a few files the data needs to be cleaned.

* Alone has instances with 2 hands and right hand in hand type that need to be removed. Likewise, spontaneous
and sync have instances with 1 hand and a mismatch between right and left hands.

In [311]:
def remove_first7(df):
    df.drop(df[df["Time"] < df["Time"].min() + 7].index, inplace=True)
    df.reset_index(drop=True, inplace=True)

In [273]:
handright.isna().sum()

Time                 0
 Frame ID            0
 Hand Type           0
 # hands             0
 Position X          0
 Position Y          0
 Position Z          0
 Velocity X          0
 Velocity Y          0
 Velocity Z          0
 Pitch               0
 Roll                0
 Yaw                 0
 Wrist Pos X       384
 Wrist Pos Y       384
 Wrist Pos Z       384
 Elbow pos X       384
 Elbow Pos Y       384
 Elbow Pos Z       384
 Grab Strenth        0
 Grab Angle          0
 Pinch Strength      0
dtype: int64

In [312]:
print("Before:", handright.shape)
handright_prepared = handright.drop(handright[(handright[" Hand Type"]=="left") | (handright[" # hands"]==2)].index)
print(handright_prepared.shape)
remove_first7(handright_prepared) # right hand file without first 7 seconds
print(handright_prepared.shape)
handright_prepared.dropna(inplace=True)
print("After:", handright_prepared.shape)
print("Null values:", handright_prepared.isna().sum().sum())
handright_prepared.head()

Before: (4812, 22)
(4698, 22)
(4277, 22)
After: (3893, 22)
Null values: 0


,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,Roll,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,135.3024,16287,right,1,11.93032,199.9861,4.981665,-35.512210,-165.3466,-57.68153,-0.092033,0.108262,-0.906612,75.45740,206.0521,45.38764,284.3946,140.0167,201.9318,0.0,0.561324,0.0
1,135.3194,16289,right,1,11.19190,195.3075,3.661716,-43.587440,-309.8374,-77.18515,-0.111674,0.126925,-0.904311,74.46141,202.4562,44.28679,282.6278,140.9712,203.6773,0.0,0.626321,0.0
2,135.3357,16291,right,1,10.53308,189.0297,2.829388,-30.403600,-363.0033,-43.79815,-0.140004,0.127219,-0.901862,73.60060,197.4387,43.54082,280.6384,142.5401,206.7526,0.0,0.636596,0.0
3,135.3524,16293,right,1,10.24300,181.9816,2.064330,-15.424000,-423.2120,-43.99948,-0.166623,0.122782,-0.901268,73.19537,191.7202,42.73155,279.7357,148.1382,209.9330,0.0,0.701644,0.0
4,135.3690,16295,right,1,10.02061,174.8592,1.228467,-8.733602,-413.4513,-53.88054,-0.197763,0.127587,-0.897777,72.68671,186.1613,41.98006,278.6890,152.5215,212.1112,0.0,0.786548,0.0


In [313]:
def combine_right(df_alone):
    left = df_alone.drop(df_alone[(df_alone[" Hand Type"]=="right") | (df_alone[" # hands"]==2)].index).reset_index(drop=True)
    if len(left) > len(handright_prepared):
        right = handright_prepared[:len(left)].copy()
    else:
        diff = len(handright_prepared) - len(left)
        left = left.drop(left[:diff].index).reset_index(drop=True)
        right = handright_prepared[:len(left)].copy()
    right.Time = left.Time
    right[" Frame ID"] = left[" Frame ID"]
    merged = right.merge(left, how="outer").sort_values("Time").reset_index(drop=True)
    merged[" # hands"] = 2
    return merged

In [ ]:
def transform_columns(df):
    # strip removes white spaces in the beginning and end
    # lower changes uppercase letters to lowercase
    df.columns = df.columns.str.strip().str.lower()
    # rename columns without spaces
    df.columns = df.columns.str.replace(" ", "_")
    # change number of hands column name
    df.columns = df.columns.str.replace("#_hands", "n_hands")

    # remove instances with 1 hand detected
    df.drop(df[df["n_hands"] == 1].index, inplace=True)

    # merge right and left hands into one row
    left = df.groupby("hand_type").get_group("left").drop(["hand_type", "n_hands", "frame_id"], axis=1)
    right = df.groupby("hand_type").get_group("right").drop(["hand_type", "n_hands", "frame_id"], axis=1)
    keep_same = {"state", "time"}
    left.columns = left.columns.map(lambda x: x if x in keep_same else x + "_left")
    right.columns = right.columns.map(lambda x: x if x in keep_same else x + "_right")
    merged = left.merge(right, how="outer", on=["state", "time"])
    return merged

In [315]:
training_data = []
validation_data = []

for filename in glob.iglob("extraFiles/Training/**/*.csv", recursive=True):
    df = pd.read_csv(filename, index_col=None, header=0)
    remove_first7(df)
    if "Alone" in filename:
        df = combine_right(df)
        df.insert(0, "state", 0)
    elif "Sync" in filename:
        df.insert(0, "state", 1)
    elif "Spontan" in filename:
        df.insert(0, "state", 2)
    df = transform_columns(df)
    training_data.append(df)

In [316]:
frame = pd.concat(training_data, axis=0, ignore_index=True)
frame.isna().sum()

state                      0
time                       0
position_x_left            0
position_y_left            0
position_z_left            0
velocity_x_left            0
velocity_y_left            0
velocity_z_left            0
pitch_left                 0
roll_left                  0
yaw_left                   0
wrist_pos_x_left           0
wrist_pos_y_left           0
wrist_pos_z_left           0
elbow_pos_x_left           0
elbow_pos_y_left           0
elbow_pos_z_left           0
grab_strenth_left          0
grab_angle_left            0
pinch_strength_left        0
position_x_right        1108
position_y_right        1108
position_z_right        1108
velocity_x_right        1108
velocity_y_right        1108
velocity_z_right        1108
pitch_right             1108
roll_right              1108
yaw_right               1108
wrist_pos_x_right       1108
wrist_pos_y_right       1108
wrist_pos_z_right       1108
elbow_pos_x_right       1108
elbow_pos_y_right       1108
elbow_pos_z_ri

In [284]:
training_data[2]

,state,time,position_x_left,position_y_left,position_z_left,velocity_x_left,velocity_y_left,velocity_z_left,pitch_left,roll_left,yaw_left,wrist_pos_x_left,wrist_pos_y_left,wrist_pos_z_left,elbow_pos_x_left,elbow_pos_y_left,elbow_pos_z_left,grab_strenth_left,grab_angle_left,pinch_strength_left,position_x_right,position_y_right,position_z_right,velocity_x_right,velocity_y_right,velocity_z_right,pitch_right,roll_right,yaw_right,wrist_pos_x_right,wrist_pos_y_right,wrist_pos_z_right,elbow_pos_x_right,elbow_pos_y_right,elbow_pos_z_right,grab_strenth_right,grab_angle_right,pinch_strength_right
0,0,525.8185,-52.96039,184.5535,-2.856696,162.15270,127.60520,-85.83782,1.238877,1.021919,1.199751,-101.04550,132.5940,4.449887,-307.7127,-12.177800,5.125422,0.0,0.073635,0.0,11.93032,199.9861,4.981665,-35.512210,-165.3466,-57.68153,-0.092033,0.108262,-0.906612,75.45740,206.0521,45.38764,284.3946,140.0167,201.9318,0.0,0.561324,0.0
1,0,525.8351,-50.04434,185.5024,-4.845364,102.82480,-24.84337,-79.46504,1.244299,1.132981,1.171960,-95.51471,131.5824,2.853924,-313.0367,3.746006,-0.704697,0.0,0.018766,0.0,11.19190,195.3075,3.661716,-43.587440,-309.8374,-77.18515,-0.111674,0.126925,-0.904311,74.46141,202.4562,44.28679,282.6278,140.9712,203.6773,0.0,0.626321,0.0
2,0,525.8516,-49.07749,182.2930,-6.442950,12.12360,-258.75260,-99.40902,1.220772,1.250050,1.152816,-94.32024,128.7403,2.742160,-322.1467,20.402740,-2.617105,0.0,0.018409,0.0,10.53308,189.0297,2.829388,-30.403600,-363.0033,-43.79815,-0.140004,0.127219,-0.901862,73.60060,197.4387,43.54082,280.6384,142.5401,206.7526,0.0,0.636596,0.0
3,0,525.8682,-50.89614,176.7708,-9.005262,-53.57292,-271.01500,-143.03330,1.096965,1.535388,1.114818,-99.72611,127.6519,5.047988,-331.5826,28.883190,-7.521224,0.0,0.149852,0.0,10.24300,181.9816,2.064330,-15.424000,-423.2120,-43.99948,-0.166623,0.122782,-0.901268,73.19537,191.7202,42.73155,279.7357,148.1382,209.9330,0.0,0.701644,0.0
4,0,525.8848,-51.76074,174.1853,-10.600340,-40.95883,-140.44390,-59.85620,1.004597,1.760037,1.091361,-102.66840,128.2257,6.734126,-333.9562,27.848030,-3.312394,0.0,0.242324,0.0,10.02061,174.8592,1.228467,-8.733602,-413.4513,-53.88054,-0.197763,0.127587,-0.897777,72.68671,186.1613,41.98006,278.6890,152.5215,212.1112,0.0,0.786548,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4098,0,593.7177,-44.79453,123.7657,-6.241947,274.44630,106.78920,125.05260,1.234747,0.282478,1.476095,-114.52440,107.5486,-6.354107,-354.6525,41.165710,45.739930,0.0,0.401728,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4099,0,593.7343,-40.77390,126.7342,-3.452266,223.52940,209.72390,160.24900,1.265178,0.283400,1.484591,-110.50080,110.7256,-3.791958,-351.3217,53.684260,55.643090,0.0,0.190267,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4100,0,593.7509,-38.56184,127.6858,-2.086977,254.93310,109.66130,157.34590,1.279585,0.280169,1.489703,-108.34430,111.9989,-2.676469,-349.8930,59.758350,58.190420,0.0,0.042665,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4101,0,593.7681,-35.88303,132.9386,0.526575,185.86870,204.98220,154.04880,1.299061,0.298048,1.489702,-105.43440,116.3043,0.041351,-346.9033,66.470160,63.191740,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


def split_train_val(df):



In [286]:
r = handright_prepared[:100].copy()
l = alone[:200]
# r.Time = l.Time
# r[" Frame ID"] = l[" Frame ID"]

diff = len(l) - len(r)
diff

# l.drop(l[:diff].index).reset_index(drop=True)
l
# merged = r.merge(l, how="outer").sort_values("Time").reset_index(drop=True)
# merged.groupby("Time")[" # hands"].sum()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,Roll,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,407.9808,47556,left,1,-179.4770,276.1585,-81.35535,99.07191,-26.663200,-51.001030,0.948318,1.034780,0.815414,-219.9878,226.1490,-53.11938,-340.6174,52.240320,79.09514,0.000000,0.920139,0.242647
1,407.9974,47557,left,1,-179.0340,276.1196,-81.67410,51.22052,-4.495606,-36.853840,0.934901,1.019857,0.801340,-219.5979,226.5593,-52.69025,-341.4207,53.470780,79.50654,0.000000,1.007900,0.269254
2,408.0139,47559,left,1,-178.1166,276.0284,-82.15177,44.13683,7.827627,-27.463290,0.913104,0.989654,0.794463,-219.2282,227.5683,-52.12496,-339.7957,56.526610,83.83327,0.000000,1.064587,0.265355
3,408.0306,47561,left,1,-177.3277,275.4324,-82.21334,57.69436,-40.209590,-1.573000,0.904349,0.943895,0.800395,-219.0942,227.8294,-51.79783,-334.6825,54.800450,85.95831,0.000000,1.174450,0.232811
4,408.0471,47563,left,1,-175.8580,273.7563,-82.06954,81.56409,-73.096860,-7.129474,0.894652,0.868642,0.837790,-219.6559,227.9204,-51.97683,-329.6036,50.930030,85.36330,0.011588,1.339837,0.191908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,411.2113,47928,left,1,-121.3494,256.9073,-92.55829,-52.93547,82.818770,21.865110,1.312111,0.469143,1.080968,-154.8785,199.6766,-71.39955,-168.8650,-9.490250,63.98888,0.000000,0.738392,0.381271
196,411.2281,47930,left,1,-121.9743,258.2573,-92.56493,-45.00289,74.893240,0.642928,1.311271,0.481302,1.089992,-156.0721,201.3614,-71.48066,-171.0412,-7.087211,64.90680,0.000000,0.708975,0.430401
197,411.2444,47932,left,1,-122.7447,259.2232,-92.58044,-43.21984,53.114780,-4.828522,1.301822,0.474176,1.078938,-157.0743,202.5940,-71.11491,-173.6659,-5.946049,64.94463,0.000000,0.744805,0.512482
198,411.2610,47934,left,1,-123.6065,260.0752,-92.82473,-56.48501,60.959110,-18.548290,1.293716,0.472095,1.062970,-157.7702,203.4537,-71.02069,-174.7301,-5.783407,63.91866,0.000000,0.778173,0.571423


In [287]:
def transform_time(df):
    # adds a column with the second
    df["start_time"] = df.time.astype(int) - df.time.min().astype(int)
    start = df.start_time.min() + 7 # remove first 7 seconds
    first_7_seconds = df[df["start_time"] < start].index
    # drop is True to prevent from adding the old indices as a new column
    df_new = df.drop(first_7_seconds).reset_index(drop=True)
    # reset index (drop = False) adds round time as the left column in the DataFrame
    df = df.groupby("start_time").nth([0, 1, 2, 3]).reset_index() # first four instances per second
    df.drop(df[df["start_time"] < df["start_time"].min() + 7].index).reset_index(drop=True)
    df.drop("start_time", axis=1, inplace=True)
    return df

alone["Time"] = alone["Time"] - alone["Time"].min()

    # reset index (drop = False) adds round time as the left column in the DataFrame
    df = df.groupby("start_time").nth([0, 1, 2, 3]).reset_index() # first four instances per second

In [180]:
# strip removes whitespaces, lower changes the uppercase letters to lowercase
sync.columns = sync.columns.str.strip().str.lower()
sync.columns

Index(['time', 'frame id', 'hand type', '# hands', 'position x', 'position y',
       'position z', 'velocity x', 'velocity y', 'velocity z', 'pitch', 'roll',
       'yaw', 'wrist pos x', 'wrist pos y', 'wrist pos z', 'elbow pos x',
       'elbow pos y', 'elbow pos z', 'grab strenth', 'grab angle',
       'pinch strength'],
      dtype='object')

In [181]:
# rename columns without spaces
sync.columns = sync.columns.str.replace(" ", "_")
# change number of hands column name
sync.columns = sync.columns.str.replace("#_hands", "n_hands")

In [182]:
sync.n_hands.unique()

array([2, 1], dtype=int64)

In [183]:
index_numbers = sync[sync["n_hands"] == 1].index

In [184]:
sync.drop(index_numbers, inplace=False)

,time,frame_id,hand_type,n_hands,position_x,position_y,position_z,velocity_x,velocity_y,velocity_z,pitch,roll,yaw,wrist_pos_x,wrist_pos_y,wrist_pos_z,elbow_pos_x,elbow_pos_y,elbow_pos_z,grab_strenth,grab_angle,pinch_strength
0,291.0203,34075,right,2,77.93928,205.6924,21.990890,-31.144050,-14.796920,124.861000,1.654635,-1.457181,-2.149903,96.24556,138.30300,12.827740,229.8921,-68.10480,65.34229,0.0,0.000000,0.0
1,291.0203,34075,left,2,-112.80230,242.7639,4.668582,5.713323,-61.391260,-21.037960,1.446623,1.405847,0.951937,-131.90310,174.49400,5.223116,-257.7061,-42.89592,16.99501,0.0,0.000000,0.0
2,291.0363,34077,right,2,76.83206,203.9094,23.754180,-85.668820,-130.091700,97.136500,1.731422,-1.463139,-2.436445,95.74135,137.59090,10.118030,231.0989,-67.86788,61.96756,0.0,0.000000,0.0
3,291.0363,34077,left,2,-113.04160,242.6686,4.462260,-14.886850,-3.294883,-9.399460,1.455516,1.406081,0.982829,-132.17560,174.41220,4.426020,-259.8255,-41.99683,14.22190,0.0,0.000000,0.0
4,291.0529,34079,right,2,74.30998,200.5741,25.159250,-148.608600,-171.942100,72.132710,1.791300,-1.455910,-2.516997,94.72282,135.58960,8.222761,233.5819,-67.46499,60.28345,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8824,365.9673,42714,left,2,-127.28110,137.6471,-55.658230,60.419170,-402.945800,99.212360,0.491071,1.811933,0.393753,-157.11800,101.79940,-3.860802,-356.9034,50.44975,136.61840,0.0,0.102073,0.0
8825,365.9842,42716,right,2,105.85730,203.8846,-40.742390,38.903880,73.382320,75.832130,1.629559,-1.190103,-1.712266,137.24730,142.04170,-45.929200,226.1846,-67.83814,55.68773,0.0,0.000000,0.0
8826,365.9842,42716,left,2,-126.16660,130.8461,-54.101770,93.572760,-388.126900,62.157910,0.482603,1.835022,0.385263,-155.38810,95.57278,-1.611514,-352.0472,50.69477,145.34620,0.0,0.095909,0.0
8827,366.0006,42718,right,2,105.98730,204.8087,-38.943860,-3.431330,47.855800,110.779000,1.689795,-1.196228,-1.848437,137.76080,143.67720,-47.348780,232.7063,-63.07462,55.24001,0.0,0.019550,0.0


In [185]:
sync.time = sync.time.round(2)

In [186]:
sync.groupby(["time", "hand_type"]).mean()

frame_id  n_hands  position_x  position_y  position_z  \
time   hand_type                                                          
291.02 left        34075.0      2.0  -112.80230    242.7639    4.668582   
       right       34075.0      2.0    77.93928    205.6924   21.990890   
291.04 left        34077.0      2.0  -113.04160    242.6686    4.462260   
       right       34077.0      2.0    76.83206    203.9094   23.754180   
291.05 left        34079.0      2.0  -113.45420    242.7294    4.221472   
...                    ...      ...         ...         ...         ...   
365.97 right       42714.0      2.0   104.87810    201.8934  -42.092990   
365.98 left        42716.0      2.0  -126.16660    130.8461  -54.101770   
       right       42716.0      2.0   105.85730    203.8846  -40.742390   
366.00 left        42718.0      2.0  -125.08190    124.3706  -53.590630   
       right       42718.0      2.0   105.98730    204.8087  -38.943860   

                  velocity_x  velocity_y  velocity_z     pitch      roll  \
time   hand_type                                                           
291.02 left         5.713323  -61.391260  -21.037960  1.446623  1.405847   
       right      -31.144050  -14.796920  124.861000  1.654635 -1.457181   
291.04 left       -14.886850   -3.294883   -9.399460  1.455516  1.406081   
       right      -85.668820 -130.091700   97.136500  1.731422 -1.463139   
291.05 left       -26.589320    4.308950  -12.123590  1.461161  1.409692   
...                      ...         ...         ...       ...       ...   
365.97 right       30.958710  105.118200  116.192400  1.562618 -1.181244   
365.98 left        93.572760 -388.126900   62.157910  0.482603  1.835022   
       right       38.903880   73.382320   75.832130  1.629559 -1.190103   
366.00 left        34.041080 -345.480500   -0.226533  0.467451  1.900445   
       right       -3.431330   47.855800  110.779000  1.689795 -1.196228   

                       yaw  wrist_pos_x  wrist_pos_y  wrist_pos_z  \
time   hand_type                                                    
291.02 left       0.951937   -131.90310    174.49400     5.223116   
       right     -2.149903     96.24556    138.30300    12.827740   
291.04 left       0.982829   -132.17560    174.41220     4.426020   
       right     -2.436445     95.74135    137.59090    10.118030   
291.05 left       0.994170   -132.38580    174.42370     3.821367   
...                    ...          ...          ...          ...   
365.97 right     -1.550808    136.04060    139.59430   -43.757560   
365.98 left       0.385263   -155.38810     95.57278    -1.611514   
       right     -1.712266    137.24730    142.04170   -45.929200   
366.00 left       0.330986   -151.16530     89.24838     0.549865   
       right     -1.848437    137.76080    143.67720   -47.348780   

                  elbow_pos_x  elbow_pos_y  elbow_pos_z  grab_strenth  \
time   hand_type                                                        
291.02 left         -257.7061    -42.89592     16.99501           0.0   
       right         229.8921    -68.10480     65.34229           0.0   
291.04 left         -259.8255    -41.99683     14.22190           0.0   
       right         231.0989    -67.86788     61.96756           0.0   
291.05 left         -260.6950    -41.63982     12.57686           0.0   
...                       ...          ...          ...           ...   
365.97 right         226.5645    -68.98385     59.13079           0.0   
365.98 left         -352.0472     50.69477    145.34620           0.0   
       right         226.1846    -67.83814     55.68773           0.0   
366.00 left         -343.8246     42.99534    152.30110           0.0   
       right         232.7063    -63.07462     55.24001           0.0   

                  grab_angle  pinch_strength  
time   hand_type                              
291.02 left         0.000000             0.0  
       right        0.000000             0.0  
291.04 left      

In [187]:
sync.head()

,time,frame_id,hand_type,n_hands,position_x,position_y,position_z,velocity_x,velocity_y,velocity_z,pitch,roll,yaw,wrist_pos_x,wrist_pos_y,wrist_pos_z,elbow_pos_x,elbow_pos_y,elbow_pos_z,grab_strenth,grab_angle,pinch_strength
0,291.02,34075,right,2,77.93928,205.6924,21.990890,-31.144050,-14.796920,124.86100,1.654635,-1.457181,-2.149903,96.24556,138.3030,12.827740,229.8921,-68.10480,65.34229,0.0,0.0,0.0
1,291.02,34075,left,2,-112.80230,242.7639,4.668582,5.713323,-61.391260,-21.03796,1.446623,1.405847,0.951937,-131.90310,174.4940,5.223116,-257.7061,-42.89592,16.99501,0.0,0.0,0.0
2,291.04,34077,right,2,76.83206,203.9094,23.754180,-85.668820,-130.091700,97.13650,1.731422,-1.463139,-2.436445,95.74135,137.5909,10.118030,231.0989,-67.86788,61.96756,0.0,0.0,0.0
3,291.04,34077,left,2,-113.04160,242.6686,4.462260,-14.886850,-3.294883,-9.39946,1.455516,1.406081,0.982829,-132.17560,174.4122,4.426020,-259.8255,-41.99683,14.22190,0.0,0.0,0.0
4,291.05,34079,right,2,74.30998,200.5741,25.159250,-148.608600,-171.942100,72.13271,1.791300,-1.455910,-2.516997,94.72282,135.5896,8.222761,233.5819,-67.46499,60.28345,0.0,0.0,0.0


In [188]:
# sync.time = sync.time.agg(lambda x: x-427)

In [189]:
# sync.set_index("Time", drop=True, inplace=False)

In [190]:
left = sync.groupby("hand_type").get_group("left").drop(["hand_type", "n_hands"], axis=1)
right = sync.groupby("hand_type").get_group("right").drop(["hand_type", "n_hands"], axis=1)

In [191]:
keep_same = {"time", "frame_id"}
left.columns = left.columns.map(lambda x: x if x in keep_same else x + "_left")
right.columns = right.columns.map(lambda x: x if x in keep_same else x + "_right")

In [192]:
merged = left.merge(right, how="outer", on=["time", "frame_id"])

#    .drop("frame_id", axis=1)
merged

,time,frame_id,position_x_left,position_y_left,position_z_left,velocity_x_left,velocity_y_left,velocity_z_left,pitch_left,roll_left,yaw_left,wrist_pos_x_left,wrist_pos_y_left,wrist_pos_z_left,elbow_pos_x_left,elbow_pos_y_left,elbow_pos_z_left,grab_strenth_left,grab_angle_left,pinch_strength_left,position_x_right,position_y_right,position_z_right,velocity_x_right,velocity_y_right,velocity_z_right,pitch_right,roll_right,yaw_right,wrist_pos_x_right,wrist_pos_y_right,wrist_pos_z_right,elbow_pos_x_right,elbow_pos_y_right,elbow_pos_z_right,grab_strenth_right,grab_angle_right,pinch_strength_right
0,291.02,34075,-112.8023,242.7639,4.668582,5.713323,-61.391260,-21.037960,1.446623,1.405847,0.951937,-131.9031,174.4940,5.223116,-257.7061,-42.89592,16.99501,0.0,0.0,0.0,77.93928,205.6924,21.99089,-31.14405,-14.796920,124.86100,1.654635,-1.457181,-2.149903,96.24556,138.3030,12.827740,229.8921,-68.10480,65.34229,0.000000,0.000000,0.000000
1,291.04,34077,-113.0416,242.6686,4.462260,-14.886850,-3.294883,-9.399460,1.455516,1.406081,0.982829,-132.1756,174.4122,4.426020,-259.8255,-41.99683,14.22190,0.0,0.0,0.0,76.83206,203.9094,23.75418,-85.66882,-130.091700,97.13650,1.731422,-1.463139,-2.436445,95.74135,137.5909,10.118030,231.0989,-67.86788,61.96756,0.000000,0.000000,0.000000
2,291.05,34079,-113.4542,242.7294,4.221472,-26.589320,4.308950,-12.123590,1.461161,1.409692,0.994170,-132.3858,174.4237,3.821367,-260.6950,-41.63982,12.57686,0.0,0.0,0.0,74.30998,200.5741,25.15925,-148.60860,-171.942100,72.13271,1.791300,-1.455910,-2.516997,94.72282,135.5896,8.222761,233.5819,-67.46499,60.28345,0.000000,0.000000,0.000000
3,291.07,34080,-113.7288,242.7941,4.114124,-31.287990,7.372077,-12.229150,1.462790,1.412944,0.991204,-132.4646,174.4369,3.603024,-260.7704,-41.65061,11.79549,0.0,0.0,0.0,73.20616,199.4709,25.63680,-125.74880,-125.683000,54.40320,1.813066,-1.450552,-2.534261,94.22473,135.0637,7.473455,235.3536,-66.46627,59.35478,0.000000,0.000000,0.000000
4,291.09,34082,-113.8567,242.1508,4.034664,-0.001188,-64.188040,6.096469,1.460808,1.417862,0.969437,-132.3376,173.7254,3.587554,-261.2905,-42.02287,10.45622,0.0,0.0,0.0,71.33848,197.9209,26.41051,-93.97961,-81.535020,42.03247,1.845923,-1.438699,-2.546650,93.46699,134.5095,6.377122,238.4372,-64.67204,56.71305,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4522,346.02,40417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-190.69900,134.7367,-73.96497,27.04534,-178.467600,107.08410,-1.693777,0.296669,-1.612239,-125.74840,159.4046,-79.341040,122.0018,125.06270,-65.98660,1.000000,2.834147,1.000000
4523,346.04,40419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-192.53520,134.0741,-73.39403,-122.46460,-33.960430,30.01225,-1.657707,0.301389,-1.599872,-127.51570,158.6437,-77.671530,119.4800,119.87770,-62.56577,1.000000,2.860157,0.912698
4524,346.05,40421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.85992,201.3093,-20.35820,199.05810,-399.482500,193.24220,0.949080,-0.916772,-0.705287,133.65540,150.2853,10.641900,255.1758,-32.04730,131.99050,0.511996,1.583867,0.000000
4525,346.07,40423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.28775,200.4698,-15.60555,128.41210,-3.501244,306.41170,0.965241,-0.918798,-0.715762,136.99070,148.4237,12.684130,279.7113,-45.45572,81.81748,0.555357,1.638591,0.000000


In [193]:
# adds a column with the second
merged["round_time"] = merged.time.astype(int) - merged.time.min().astype(int)

In [194]:
# reset index (drop = False) adds round time as the left column in the DataFrame
merged = merged.groupby("round_time").nth([0, 1, 2, 3]).reset_index()

In [195]:
# drop is True to prevent from adding the old indices as a new column
merged = merged.drop(merged[merged["round_time"] < merged.round_time.min() + 7].index).reset_index(drop=True)

In [ ]:
merged

In [86]:
round_time_groups = merged.groupby("round_time")

### Notes:

01:37:00

- Organize the data in a pandas dataframe
- Goal: using the hand ... , detect if is alone, sync or spontaneous.
- "spontaneous synchronizing"
- interpersonal space
- training has 9 participants
- validation has different participants
- spontan and sync: if # hands is 1, remove data!
- time series
- every 2 lines is one feature (need to be combined) = 1 frame
- choose how many frames
- position y is similar
- 2 recordings, second is usually better
- can't split train and test as usual (correlation between each following movement). can't shuffle!
for instance, can take first 40 seconds for training and last 5 for testing.
- at least 2 seconds in between train and test
- model for 1 person, try model on second person. train the second person and test the third etc.
- 4 frames per second
-

> #### Resources:
1. Leap Motion Attributes <a href="https://developer-archive.leapmotion.com/documentation/python/api/Leap.Hand.html"
> title="leapmotion">link</a>

In [ ]:
training_data = []
validation_data = []
training_path = "extraFiles/Training"
training_dirs = glob.glob(training_path + "/*")
# seed = 0

for dir in training_dirs:
    training_files = glob.glob(dir + "/*.csv")
    for filename in training_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        if "Alone" in filename:
            df = combine_right(df)
            df.insert(0, "state", 0)
        elif "Sync" in filename:
            df.insert(0, "state", 1)
        elif "Spontan" in filename:
            df.insert(0, "state", 2)
        df = transform_columns(df)
        training_data.append(df)

# frame = pd.concat(training_data, axis=0, ignore_index=True)